Lecture 3: Multi-Table SQL
======================

Joins!
------
Let's illustrate some more complex queries that join two tables together.

* Consider a table of companies, stock price, and HQ country.
> company(<u>cname</u>, stockprice, country)
 
* we'll then revist products and introduce some consistency requirements 

In [4]:
%%sql
drop table if exists product; -- This needs to be dropped if exists, see why further down!
drop table if exists company;
create table company (
    cname varchar primary key, -- company name uniquely identifies the company.
    stockprice money, -- stock price is in money 
    country varchar); -- country is just a string
insert into company values ('GizmoWorks', 25.0, 'USA');
insert into company values ('Canon', 65.0, 'Japan');
insert into company values ('Hitachi', 15.0, 'Japan');

Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [5]:
%sql select * from company;

Done.


cname,stockprice,country
GizmoWorks,25,USA
Canon,65,Japan
Hitachi,15,Japan


Foreign Key Constraints
-----------------------
* Suppose that we want to create a products table

> Product(pname, price, category, manufacturer)

* Something is odd here: We can have manufacturers that sell products but don't occur in our company table!
* To protect against, this we introduce _foreign keys_ 

We say the company name in products _refers_ to cname in company. Let's do it in SQL! the key statement below is:

> foreign key (manufacturer) references company(cname)

  * Note that cname must be a key in company! 
  * Keys and Foreign keys come up _all_ the time. 
    * PKs and FKs are common (others, less so)

In [6]:
%%sql drop table if exists product;
pragma foreign_keys = ON; -- WARNING by default off in sqlite
create table product(
       pname varchar primary key, -- name of the product
       price money, -- price of the product
       category varchar, -- category
       manufacturer varchar, -- manufacturer
       foreign key (manufacturer) references company(cname));

insert into product values('Gizmo', 19.99, 'Gadgets', 'GizmoWorks');
insert into product values('PowerGizmo', 29.99, 'Gadgets', 'GizmoWorks');
insert into product values('SingleTouch', 149.99, 'Photography', 'Canon');
insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');

Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Indeed foreign keys are a _constraint_ 
> What happens if we introduce a company name not in our table?


In [7]:
try:
    %sql insert into product values('MultiTouch', 203.99, 'Household', 'Google');
except Exception as e:
    print e
    print "Rejected!"

(IntegrityError) UNIQUE constraint failed: product.pname u"insert into product values('MultiTouch', 203.99, 'Household', 'Google');" ()
Rejected!


In [8]:
%%sql
-- the update is rejected!
select * from product;

Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon
MultiTouch,203.99,Household,Hitachi


Foreign Keys and Delete
=============

* What happens if we delete a company? Three options:
  * Disallow the delete. (default)
  * Remove all products (add "`on delete cascade`")
  * A third variant due to NULL
  

**First option (default)- delete is disallowed**

In [10]:
try:
    %sql delete from company where cname = 'Hitachi';
except Exception as e:
    print e
    print "Disallowed!"

(IntegrityError) FOREIGN KEY constraint failed u"delete from company where cname = 'Hitachi';" ()
Disallowed!


**Second option: remove all products belonging to the company we delete**

Try adjusting the foreign key constraint clause when you create the products table as follows:
> foreign key (manufacturer) references company(cname) on delete cascade

Now, when a company row is deleted, all of the products linked by the foreign key constraint will be deleted as well.

Joins!
------
> Product (<u>pname</u>,  price, category, manufacturer)<br>
> Company (<u>cname</u>, stockPrice, country)

The query we want to answer is:

> Find all products under $200 manufactured in Japan;
> return their names and prices. 

Notice products don't have a location and manufacturers don't have price. Need info in _each_ of the tables.

In [11]:
%%sql
SELECT pname, price
FROM product, company
where manufacturer=cname and country='Japan' and price <= 200;

Done.


pname,price
SingleTouch,149.99


Let's see how to write this join in a modular way.

In [12]:
%%sql -- Part 1: Set of Japanese companies.
SELECT distinct cname -- do we need distinct?
from company where country='Japan';

Done.


cname
Canon
Hitachi


In [13]:
%%sql -- Part 2: Cheap Products (under $200)
select distinct pname, price, manufacturer
from product
where price <= 200;

Done.


pname,price,manufacturer
Gizmo,19.99,GizmoWorks
PowerGizmo,29.99,GizmoWorks
SingleTouch,149.99,Canon


In [14]:
%%sql -- combine them with nested SFW queries... this is a cross product?
SELECT * 
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 200) CheapProducts,
  (SELECT DISTINCT cname
   FROM company
   WHERE country='Japan') JapaneseProducts;

Done.


pname,price,manufacturer,cname
Gizmo,19.99,GizmoWorks,Canon
Gizmo,19.99,GizmoWorks,Hitachi
PowerGizmo,29.99,GizmoWorks,Canon
PowerGizmo,29.99,GizmoWorks,Hitachi
SingleTouch,149.99,Canon,Canon
SingleTouch,149.99,Canon,Hitachi


In [15]:
%%sql
-- Combine them as a join!
SELECT DISTINCT pname, price
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 200) CheapProducts,
  (SELECT distinct cname
   FROM company
   WHERE country='Japan') JapaneseProducts
WHERE cname = manufacturer;

Done.


pname,price
SingleTouch,149.99


Takeways
--------
* There are potentially _many logically equivalent ways_ to write a query
    * This fact will be used later by the query optimizer and in homework!
    * On exams, write the simplest thing (break it down in parts?)

Duplicate answers from join
--------------------------

Note that we can get duplicate answers from a join...

In [16]:
%%sql -- duplicate answer
SELECT Country
FROM Product, Company
WHERE  Manufacturer=CName AND Category='Gadgets';

Done.


country
USA
USA


NOTE:
-----

Please run the below cells first before proceeding- you'll need them soon!

In [2]:
%%sql
pragma foreign_keys = ON; -- WARNING: by default off in sqlite
drop table if exists product; -- This needs to be dropped if exists, see why further down!
drop table if exists company;
create table company (
    cname varchar primary key, -- company name uniquely identifies the company.
    stockprice money, -- stock price is in money 
    country varchar); -- country is just a string
insert into company values ('GizmoWorks', 25.0, 'USA');
insert into company values ('Canon', 65.0, 'Japan');
insert into company values ('Hitachi', 15.0, 'Japan');
create table product(
       pname varchar primary key, -- name of the product
       price money, -- price of the product
       category varchar, -- category
       manufacturer varchar, -- manufacturer
       foreign key (manufacturer) references company(cname));
insert into product values('Gizmo', 19.99, 'Gadgets', 'GizmoWorks');
insert into product values('SingleTouch', 149.99, 'Photography', 'Canon');
insert into product values('PowerGizmo', 29.99, 'Gadgets', 'GizmoWorks');
insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');

Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [3]:
%%sql
DROP TABLE IF EXISTS franchise;
CREATE TABLE franchise (name TEXT, db_type TEXT);
INSERT INTO franchise VALUES ('Bobs Bagels', 'NoSQL');
INSERT INTO franchise VALUES ('eBagel', 'NoSQL');
INSERT INTO franchise VALUES ('BAGEL CORP', 'MySQL');

DROP TABLE IF EXISTS store;
CREATE TABLE store (franchise TEXT, location TEXT);
INSERT INTO store VALUES ('Bobs Bagels', 'NYC');
INSERT INTO store VALUES ('eBagel', 'PA');
INSERT INTO store VALUES ('BAGEL CORP', 'Chicago');
INSERT INTO store VALUES ('BAGEL CORP', 'NYC');
INSERT INTO store VALUES ('BAGEL CORP', 'PA');

DROP TABLE IF EXISTS bagel;
CREATE TABLE bagel (name TEXT, price MONEY, made_by TEXT);
INSERT INTO bagel VALUES ('Plain with shmear', 1.99, 'Bobs Bagels');
INSERT INTO bagel VALUES ('Egg with shmear', 2.39, 'Bobs Bagels');
INSERT INTO bagel VALUES ('eBagel Drinkable Bagel', 27.99, 'eBagel');
INSERT INTO bagel VALUES ('eBagel Expansion Pack', 1.99, 'eBagel');
INSERT INTO bagel VALUES ('Plain with shmear', 0.99, 'BAGEL CORP');
INSERT INTO bagel VALUES ('Organic Flax-seed bagel chips', 0.99, 'BAGEL CORP');

DROP TABLE IF EXISTS purchase;
-- Note that date is an int here just to simplify things
CREATE TABLE purchase (bagel_name TEXT, franchise TEXT, date INT, quantity INT, purchaser_age INT);
INSERT INTO purchase VALUES ('Plain with shmear', 'Bobs Bagels', 1, 12, 28);
INSERT INTO purchase VALUES ('Egg with shmear', 'Bobs Bagels', 2, 6, 47);
INSERT INTO purchase VALUES ('Plain with shmear', 'BAGEL CORP', 2, 12, 24);
INSERT INTO purchase VALUES ('Plain with shmear', 'BAGEL CORP', 3, 1, 17);
INSERT INTO purchase VALUES ('eBagel Expansion Pack', 'eBagel', 1, 137, 5);
INSERT INTO purchase VALUES ('Plain with shmear', 'Bobs Bagels', 4, 24, NULL);

Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Order By Reprise
================
* SQL-89 forbids the following, but we can make sense of it:
> SELECT pname FROM Product ORDER BY Price

Let's try out some other slightly unintuitive ORDER BY queries:

In [4]:
%sql SELECT * FROM Product;

Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon
PowerGizmo,29.99,Gadgets,GizmoWorks
MultiTouch,203.99,Household,Hitachi


In [5]:
%%sql SELECT pname,price FROM Product
ORDER BY pname

Done.


pname,price
Gizmo,19.99
MultiTouch,203.99
PowerGizmo,29.99
SingleTouch,149.99


In [6]:
%%sql SELECT pname FROM Product
ORDER BY Price

Done.


pname
Gizmo
PowerGizmo
SingleTouch
MultiTouch


In [7]:
%%sql SELECT distinct pname FROM Product
ORDER BY Price

Done.


pname
Gizmo
PowerGizmo
SingleTouch
MultiTouch


* Some engines- like Postgres- would balk at this, but some engines- like SQLite- do process this... how?